# Mi cadena de markov

## Bibliotecas

In [1]:
# Warnings ---------------------------------------------------------------------------------------------------

import warnings
warnings.filterwarnings("ignore")

# Lectura y manipulación de datos ----------------------------------------------------------------------------

import pandas as pd

# Descomposición espectral -----------------------------------------------------------------------------------

import numpy as np
from numpy.linalg import eig, inv


## Lectura de la base de datos

In [2]:
data = pd.read_parquet('data/tec_estocasticos.parquet', engine='pyarrow')

In [3]:
data

,periodo,cliente_id,material_id,tipo_cliente
0,05-2022,4894.0,22.0,Distribuidor
1,05-2022,4769.0,17.0,Distribuidor
2,05-2022,4823.0,227.0,Distribuidor
3,08-2022,4816.0,340.0,Distribuidor
4,08-2022,4888.0,270.0,Distribuidor
...,...,...,...,...
6973895,05-2022,1816.0,1861.0,Farmacia
6973896,05-2022,1725.0,1373.0,Hospital
6973897,05-2022,588.0,2249.0,Hospital
6973898,05-2022,565.0,191.0,Hospital


In [ ]:
data['periodo'] = pd.to_datetime(data['periodo'])
data.sort_values(by='periodo', inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis = 1,inplace=True)
data

### Tipos de Cliente

In [ ]:
data['tipo_cliente'].unique()

## Función para obtener la matriz de transición de un cliente comprando un determinado producto

In [ ]:
def matriz_transicion(tipo_cliente, cliente_id, material_id):
    
    # Subset
    cliente_tipo = data.loc[data['tipo_cliente'] == tipo_cliente]
    pruducto = cliente_tipo.loc[cliente_tipo['material_id'] == material_id]
    id_cliente = pruducto.loc[pruducto['cliente_id'] == cliente_id]
    id_cliente.reset_index(inplace = True)
    id_cliente.drop('index', axis = 1, inplace = True)
    
    # Estados: Compró o no compró
    
    t = [0]

    for x in range(0, len(id_cliente['periodo'])-2):
        if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
            t.append(0)
        else:
            for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
                t.append(1)
            t.append(0)
    
    t_1 = []

    for x in range(0, len(id_cliente['periodo'])-1):
        if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
            t_1.append(0)
        else:
            for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
                t_1.append(1)
            t_1.append(0)
            

    estados = pd.DataFrame()

    estados['t'] = t
    estados['t_1'] = t_1
    
    Xt = estados['t'][0:-1].reset_index(drop=True).rename('X_t')
    Xt_1 = estados['t_1'][1::].reset_index(drop=True).rename('X_t+1')
    
    new_data=pd.concat((Xt, Xt_1), axis=1)
    
    matriz_transicion = new_data.groupby('X_t').value_counts(normalize=True).unstack(level='X_t+1')
    matriz_transicion= matriz_transicion.fillna(0)
    
    return matriz_transicion

In [ ]:
 matriz_transicion('Distribuidor', 4769.0, 17.0)

In [ ]:
matriz_transicion =  matriz_transicion('Distribuidor', 4769.0, 17.0)

## Descomposición espectral

In [ ]:
## Descomposición espectral

import numpy as np
from numpy.linalg import eig, inv
Lambda, Q = eig(matriz_transicion)
print("Los eigenvalores de P son:", Lambda)
print("Los eigenvectores de P son:", Q)

In [ ]:
Q_1 = inv(Q)
Lambda = np.diag(Lambda)
PP = np.matmul(np.matmul(Q, Lambda), Q_1)
PP.round(decimals = 4)

## Consultar probabilidad en tiempo n

In [ ]:
def pasos(n):
    Lambda_n = Lambda**n
    P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)
    
    df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
    df.rename(columns = {0:'Compra'}, inplace = True)
    df.rename(columns = {1:'No Compra'}, inplace = True)
    return df

In [ ]:
pasos(5)

In [ ]:
pasos(10)

In [ ]:
pasos(15)

In [ ]:
data.loc[data['tipo_cliente'] == 'Hospital']

## Dropdown

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# ------

tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = {}

for x in tipo_cliente:
        cliente_id[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())

boton1 = widgets.Dropdown(options=tipo_cliente, description='Tipo cliente:')
boton2 = widgets.Dropdown(options=[], description='Cliente id:')


def actualizar_opciones(*args):
    boton2.options = cliente_id[boton1.value]

boton1.observe(actualizar_opciones, 'value')


# -----

material_id = list(data['material_id'].unique()) 
      
boton3 = widgets.Dropdown(options =material_id, description='Material id:')

pasos_t = list(range(1,51))

boton4 = widgets.Dropdown(options= pasos_t, description='Pasos:')


# Función para la visualización con parámetros
def valores(tipo_cliente, cliente_id, material_id, pasos_t):
    
    try: 
        Lambda, Q = eig(matriz_transicion(tipo_cliente, cliente_id, material_id))
        Q_1 = inv(Q)
        Lambda = np.diag(Lambda)
        PP = np.matmul(np.matmul(Q, Lambda), Q_1)

        Lambda_n = Lambda**pasos_t
        P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)

        df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
        df.rename(columns = {0:'Compra'}, inplace = True)
        df.rename(columns = {1:'No Compra'}, inplace = True)
    
    except:
        return print('No existe información de ese cliente comprando ese producto')
    return print(df)


widgets.interactive(valores, 
                    tipo_cliente=boton1, 
                    cliente_id=boton2,
                    material_id=boton3,
                    pasos_t = boton4)


In [ ]:
tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = {}

for x in tipo_cliente:
    cliente_id[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())
        
        
cliente_id

In [ ]:
material_id = {}

for x in tipo_cliente:
    for i in cliente_id[x]:
        material_id[i] = list(data.loc[data['cliente_id'] == i]['material_id'].unique())

In [ ]:
material_id

In [ ]:
cliente_id['Hospital']

In [ ]:
# Markdown Funcional

# Opciones

tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = list(data['cliente_id'].unique()) 

material_id = list(data['material_id'].unique()) 

pasos_t = list(range(1,51))

# Botones

dropdown_tipo_cliente = widgets.Dropdown(
    options=tipo_cliente,
    description="tipo_cliente:"
)

dropdown_cliente_id = widgets.Dropdown(
    options=cliente_id,
    description="cliente_id:"
)

dropdown_material_id = widgets.Dropdown(
    options= material_id,
    description="material_id:"
)

dropdown_pasos = widgets.Dropdown(
    options=pasos_t,
    description="pasos:"
)


# Función para la visualización con parámetros
def valores(tipo_cliente, cliente_id, material_id, pasos_t):
    
    try: 
        Lambda, Q = eig(matriz_transicion(tipo_cliente, cliente_id, material_id))
        Q_1 = inv(Q)
        Lambda = np.diag(Lambda)
        PP = np.matmul(np.matmul(Q, Lambda), Q_1)

        Lambda_n = Lambda**pasos_t
        P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)

        df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
        df.rename(columns = {0:'Compra'}, inplace = True)
        df.rename(columns = {1:'No Compra'}, inplace = True)
    
    except:
        return print('No existe información de ese cliente comprando ese producto')
    return print(df)

widgets.interactive(valores, 
                    tipo_cliente=dropdown_tipo_cliente, 
                    cliente_id=dropdown_cliente_id,
                    material_id=dropdown_material_id,
                    pasos_t = dropdown_pasos)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define las opciones iniciales para el primer y segundo botón
opciones_boton1 = ['Opción A', 'Opción B', 'Opción C']
opciones_boton2 = {
    'Opción A': ['A1', 'A2'],
    'Opción B': ['B1', 'B2', 'B3'],
    'Opción C': ['C1', 'C2', 'C3']
}

opciones_boton1 = list(data['tipo_cliente'].unique())

opciones_boton2 = {}

for x in opciones_boton1:
        opciones_boton2[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())

# Crea los widgets para los botones desplegables
boton1 = widgets.Dropdown(options=opciones_boton1, description='Botón 1:')
boton2 = widgets.Dropdown(options=[], description='Botón 2:')

# Función para actualizar las opciones del botón 2 según la selección en el botón 1
def actualizar_opciones(*args):
    boton2.options = opciones_boton2[boton1.value]

# Vincula la función de actualización al evento de cambio en el botón 1
boton1.observe(actualizar_opciones, 'value')

# Muestra los widgets
display(boton1, boton2)


In [ ]:
opciones_boton2

In [ ]:
tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = {}

for x in tipo_cliente:
    cliente_id[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())
    
material_id = {}

for x in cliente_id.keys():
    for i in cliente_id[x]:
        material_id[i] = list(data.loc[data['cliente_id'] == i]['material_id'].unique())

In [ ]:
material_id

In [ ]:
cliente_id.values()

In [ ]:
material_id

In [4]:
data.to_csv('data.csv')